In [6]:
# in layers tensorlow req 
import os
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, concatenate, Cropping2D, Conv2DTranspose
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping
from PIL import Image, UnidentifiedImageError
from skimage.metrics import peak_signal_noise_ratio as psnr, mean_squared_error as mse
from sklearn.metrics import mean_absolute_error as mae
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split

img_size=256
# img_size=128
# img_size=64
def load_images(image_dir, image_size=(img_size, img_size)):
    images = []
    for img_name in sorted(os.listdir(image_dir)):
        img_path = os.path.join(image_dir, img_name)
        img = load_img(img_path, target_size=image_size)
        img = img_to_array(img) /255
        images.append(img)
       
    return np.array(images)



In [7]:
def unet(input_shape=(img_size,img_size,3)):
    inputs=Input(input_shape)
    
    #encode
    conv1=Conv2D(32,3,activation='relu',padding='same')(inputs)
    conv1=Conv2D(32,3,activation='relu',padding='same')(conv1)
    pool1=MaxPooling2D(pool_size=(2,2))(conv1)

    conv2=Conv2D(64,3,activation='relu',padding='same')(pool1)
    conv2=Conv2D(64,3,activation='relu',padding='same')(conv2)
    pool2=MaxPooling2D(pool_size=(2,2))(conv2)

    conv3=Conv2D(128,3,activation='relu',padding='same')(pool2)
    conv3=Conv2D(128,3,activation='relu',padding='same')(conv3)
    pool3=MaxPooling2D(pool_size=(2,2))(conv3)

    # base part
    conv4=Conv2D(256,3,activation='relu',padding='same')(pool3)
    conv4=Conv2D(256,3,activation='relu',padding='same')(conv4)

    # decoder
    up5=Conv2DTranspose(128,(2,2),strides=(2,2),padding='same')(conv4)
    up5=concatenate([up5,conv3],axis=-1)
    conv5=Conv2D(128,3,activation='relu',padding='same')(up5)
    conv5=Conv2D(128,3,activation='relu',padding='same')(conv5)

    up6=Conv2DTranspose(64,(2,2),strides=(2,2),padding='same')(conv5)
    up6=concatenate([up6,conv2],axis=-1)
    conv6=Conv2D(64,3,activation='relu',padding='same')(up6)
    conv6=Conv2D(64,3,activation='relu',padding='same')(conv6)

    up7=Conv2DTranspose(32,(2,2),strides=(2,2),padding='same')(conv6)
    up7=concatenate([up7,conv1],axis=-1)
    conv7=Conv2D(32,3,activation='relu',padding='same')(up7)
    conv7=Conv2D(32,3,activation='relu',padding='same')(conv7)

    outputs=Conv2D(3,1,activation='sigmoid')(conv7)
    # linear is somehow worse search why

    model=Model(inputs=[inputs],outputs=[outputs])
    return model


In [8]:
def train_model():
    clean_images=load_images('./Train/high')
    noisy_images=load_images('./Train/low')
    train_noisy,val_noisy,train_clean,val_clean=train_test_split(noisy_images,clean_images,test_size=0.1,random_state=42)




    model=unet(input_shape=(img_size,img_size,3))

    datagen=ImageDataGenerator(
        rotation_range=30,
        width_shift_range=0.1,
        height_shift_range=0.1,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        vertical_flip=True,
        fill_mode='nearest'
    )
    
    train_generator=datagen.flow(train_noisy,train_clean,batch_size=32)
    val_generator=datagen.flow(val_noisy,val_clean,batch_size=32)

    model.compile(optimizer=Adam(learning_rate=0.001),loss='mean_squared_error')
    # used rmsprop  


    early_stopping=EarlyStopping(monitor='val_loss',patience=10,restore_best_weights=True)
    model.fit(train_generator,epochs=10,validation_data=val_generator,callbacks=[early_stopping])

    return model


In [4]:

#mention source of this part code 
def evaluate_model(model, clean_images, noisy_images):
    predictions = model.predict(noisy_images)
    mse_scores = [mse(clean, pred) for clean, pred in zip(clean_images, predictions)]
    psnr_scores = [psnr(clean, pred) for clean, pred in zip(clean_images, predictions)]
    mae_scores = [mae(clean.flatten(), pred.flatten()) for clean, pred in zip(clean_images, predictions)]
    
    print(f"Mean MSE: {np.mean(mse_scores)}")
    print(f"Mean PSNR: {np.mean(psnr_scores)}")
    print(f"Mean MAE: {np.mean(mae_scores)}")


In [5]:
# sover guy this calles only if main 
if __name__ == '__main__':
    model = train_model()
    
    clean_images =load_images('./Train/high')
    noisy_images=load_images('./Train/low')

    evaluate_model(model, clean_images, noisy_images)

Epoch 1/10


C:\Users\kunda\AppData\Roaming\Python\Python311\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:120: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


14/14 ━━━━━━━━━━━━━━━━━━━━ 99s 7s/step - loss: 0.0503 - val_loss: 0.0491
Epoch 2/10
14/14 ━━━━━━━━━━━━━━━━━━━━ 81s 6s/step - loss: 0.0490 - val_loss: 0.0483
Epoch 3/10
14/14 ━━━━━━━━━━━━━━━━━━━━ 81s 6s/step - loss: 0.0464 - val_loss: 0.0485
Epoch 4/10
14/14 ━━━━━━━━━━━━━━━━━━━━ 83s 6s/step - loss: 0.0480 - val_loss: 0.0489
Epoch 5/10
14/14 ━━━━━━━━━━━━━━━━━━━━ 94s 7s/step - loss: 0.0473 - val_loss: 0.0483
Epoch 6/10
14/14 ━━━━━━━━━━━━━━━━━━━━ 92s 6s/step - loss: 0.0488 - val_loss: 0.0481
Epoch 7/10
14/14 ━━━━━━━━━━━━━━━━━━━━ 88s 6s/step - loss: 0.0467 - val_loss: 0.0482
Epoch 8/10
14/14 ━━━━━━━━━━━━━━━━━━━━ 80s 6s/step - loss: 0.0480 - val_loss: 0.0475
Epoch 9/10
14/14 ━━━━━━━━━━━━━━━━━━━━ 80s 6s/step - loss: 0.0473 - val_loss: 0.0478
Epoch 10/10
14/14 ━━━━━━━━━━━━━━━━━━━━ 81s 6s/step - loss: 0.0467 - val_loss: 0.0475
16/16 ━━━━━━━━━━━━━━━━━━━━ 18s 1s/step
Mean MSE: 0.04198443314171405
Mean PSNR: 14.21472389408663
Mean MAE: 0.16976924240589142
3/3 ━━━━━━━━━━━━━━━━━━━━ 3s 737ms/step


In [9]:
# saving model not doing again
model.save('advdsv.h5')

print("Model saved as 'advdsv.h5'")

Model saved as 'denoising_model.h5'
